In [ ]:
# Importing modules

from dotenv import load_dotenv
import os
import pandas as pd
import requests

load_dotenv(dotenv_path="E:\\PROJECTS\\LEAGUE BOT\\.env")
api_key = os.environ.get('riot_apikey')
# print(api_key)

RGAPI-5c810d00-8382-43d8-a6f5-fecc758d3cc7


In [43]:
# Functions

# Get PUUID from IGN
def get_puuid(game_name=None, tag_line=None, api_key=None):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}"
    response = requests.get(link)
    return response.json()["puuid"]

# Get IGN from PUUID
def get_ign(puuid=None):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
    response = requests.get(link)
    game_name = response.json()["gameName"]
    tag_line = response.json()["tagLine"]
    return f'{game_name}#{tag_line}'

# Get List of Matches by PUUID
def get_matchhistory(region=None, puuid=None):
    link = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={api_key}"
    # For the part of link that is "start=0&count=20", I can edit this to increase the count to the limit of 100.
    response = requests.get(link)
    return response.json()

# Get match information form Match ID
def get_matchdata(region=None, match_id=None):
    link = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    response = requests.get(link)
    return response.json()

# Function to Extract from json
def json_extract(obj, key):

    arr = []
    def extract(obj, arr, key):    
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr    

    values = extract(obj, arr, key)
    return values


In [44]:
# my puuid
# lh_MFV9FRF3M2Apr3J6jnW80PiQKoJoc3SHJCJ_Jk_Y2WU-Iao60QBwtmeXXVTyFWDuHJ_YLizhQWQ

In [45]:
# Processing a Match

def process_match_json(match_json, puuid):
    
    # Game Data (Root)
    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']
    match_id = metadata['matchId']
    participants = metadata['participants']
    player = players[participants.index(puuid)]
    teams = info['teams']
    perks = player["perks"]
    stats = perks["statPerks"]
    styles = perks["styles"]

    primary = styles[0]
    secondary = styles[1]

    # Game Results & Timings:
    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_time_stamp = info['gameEndTimestamp']
    patch = info['gameVersion']
    result = player['win']
    ifearly_surrender = player['gameEndedInEarlySurrender']
    if_surrender = player['gameEndedInSurrender']

    # Player Stats
    assists = player['assists']
    kills = player['kills']
    deaths = player['deaths']
    minion_cs = player['totalMinionsKilled']
    jgl_cs = player['neutralMinionsKilled'] # for junglers
    total_cs = minion_cs + jgl_cs

    # Economy
    gold_earned = player['goldEarned']
    gold_spent = player['goldSpent']
    items_purchased = player["itemsPurchased"]
    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    # Pings
    allin_ping = player['allInPings']
    assistme_ping = player['assistMePings']
    enemymissing_ping = player['enemyMissingPings']
    enemyvision_ping = player['enemyVisionPings']
    needvision_ping = player['needVisionPings']
    retreat_ping = player['getBackPings']
    defend_ping = player['holdPings']
    coming_ping = player['onMyWayPings']
    push_ping = player['pushPings']
    blue_ping = player['commandPings']
    all_pings = allin_ping + assistme_ping + enemymissing_ping + needvision_ping + retreat_ping + defend_ping + coming_ping + push_ping + blue_ping 

    # Sums & Spells
    spell1_casts = player['spell1Casts']
    spell2_casts = player['spell2Casts']
    spell3_casts = player['spell3Casts']
    spell4_casts = player['spell4Casts']
    summoner1_casts = player['summoner1Casts']
    summoner1_id = player['summoner1Id']
    summoner2_casts = player['summoner2Casts']
    summoner2_id = player['summoner2Id']

    # Vision Stuff
    wards_killed = player['wardsKilled']
    wards_placed = player['wardsPlaced']
    sight_wards = player['sightWardsBoughtInGame']
    vision_wards = player['visionWardsBoughtInGame']
    detector_wards = player['detectorWardsPlaced']
    vision_score = player['visionScore']

    # Damage Stats

    # Dealt
    total_damagedealt = player["totalDamageDealt"]
    total_damagedealttochampions = player["totalDamageDealtToChampions"]
    physical_damagedealt = player["physicalDamageDealt"]
    physical_damagedealttochampions = player["physicalDamageDealtToChampions"]
    magic_damagedealt = player ["magicDamageDealt"]
    magic_damagedealttochampions = player ["magicDamageDealtToChampions"]
    damagedealt_toobjectives = player["damageDealtToObjectives"]
    damagedealt_toturrets = player["damageDealtToTurrets"]
    true_damagedealt = player["trueDamageDealt"]
    true_damagedealttochampions = player["trueDamageDealtToChampions"]

    # Taken
    total_damagetaken = player["totalDamageTaken"]
    physical_damagetaken = player["physicalDamageTaken"]
    magic_damagetaken = player["magicDamageTaken"]
    true_damagetaken = player["trueDamageTaken"]

    # Heals & Shields
    damage_selfmitigated = player["damageSelfMitigated"]
    total_damagehieldedonteammates= player["totalDamageShieldedOnTeammates"]
    total_heal = player["totalHeal"]
    total_healsonteammates = player["totalHealsOnTeammates"]

    # Time
    time_ccothers = player ["timeCCingOthers"]
    total_timespentdead = player ["totalTimeSpentDead"]
    longest_timespentliving = player ["longestTimeSpentLiving"]

    # Extra Stats

    # Objectives
    dragon_kills = player["dragonKills"]
    obj_stolen = player["objectivesStolen"]
    obj_stolenass = player["objectivesStolenAssists"]
    baron_kills = player["baronKills"]
    first_towerkill = player["firstTowerKill"]
    first_towerass = player["firstTowerAssist"]
    inhib_kills = player["inhibitorKills"]
    inhib_lost = player["inhibitorsLost"]

    # Killing Sprees
    first_bloodkills = player["firstBloodKill"]
    firstblood_assist = player["firstBloodAssist"]
    killing_sprees = player["killingSprees"]
    largest_killingsprees = player["largestKillingSpree"]
    triple_kills = player["tripleKills"]
    quadra_kills = player["quadraKills"]
    penta_kills = player["pentaKills"]

    # Runes

    defense = stats["defense"]
    offense = stats["offense"]
    flex = stats["flex"]

    primary_style = primary["style"]
    secondary_style = secondary["style"]

    primary_keystone = primary["selections"][0]["perk"]
    primary_perk_1 = primary["selections"][1]["perk"]
    primary_perk_2 = primary["selections"][2]["perk"]
    primary_perk_3 = primary["selections"][3]["perk"]

    secondary_perk_1 = secondary["selections"][0]["perk"]
    secondary_perk_2 = secondary["selections"][1]["perk"]

    # Team Stats
    for team in teams:
        if team["teamId"] == player["teamId"]:
            objectives = team["objectives"]
            baron = objectives["baron"]
            dragon = objectives["dragon"]
            grubs = objectives["horde"]
            rift_herald = objectives["riftHerald"]
            tower = objectives["tower"]
            inhibitor = objectives["inhibitor"]

    # for objectives in [baron, dragon, grubs, rift_herald, tower, inhibitor]:
        # first = objectives["first"]
        # objective_kills = objectives["kills"]

    # DataFrame

    match_df = pd.DataFrame({
        "match_id" : [match_id],
        "participants" : [participants],
        "game_creation" : [game_creation],
        "game_duration" : [game_duration],
        "game_start_timestamp" : [game_start_timestamp],
        "game_end_time_stamp" : [game_end_time_stamp],
        "patch" : [patch],
        "result" : [result],
        "ifearly_surrender" : [ifearly_surrender],
        "if_surrender" : [if_surrender],
        "assists" : [assists],
        "kills" : [kills],
        "deaths" : [deaths],
        "minion_cs" : [minion_cs],
        "jgl_cs" : [jgl_cs],
        "total_cs" : [total_cs],    
        "gold_earned" : [gold_earned], 
        "gold_spent" : [gold_spent], 
        "items_purchased" : [items_purchased], 
        "item0" : [item0], 
        "item1" : [item1], 
        "item2" : [item2], 
        "item3" : [item3], 
        "item4" : [item4], 
        "item5" : [item5], 
        "item6" : [item6], 
        "allin_ping": [allin_ping],
        "assistme_ping": [assistme_ping],
        "enemymissing_ping": [enemymissing_ping],
        "enemyvision_ping": [enemyvision_ping],
        "needvision_ping": [needvision_ping],
        "retreat_ping": [retreat_ping],
        "defend_ping": [defend_ping],
        "coming_ping": [coming_ping],
        "push_ping": [push_ping],
        "blue_ping": [blue_ping],
        "all_pings": [all_pings],
        "spell1_casts": [spell1_casts],
        "spell2_casts": [spell2_casts],
        "spell3_casts": [spell3_casts],
        "spell4_casts": [spell4_casts],
        "summoner1_casts": [summoner1_casts],
        "summoner1_id": [summoner1_id],
        "summoner2_casts": [summoner2_casts],
        "summoner2_id": [summoner2_id],
        "wards_killed": [wards_killed],
        "wards_placed": [wards_placed],
        "sight_wards": [sight_wards],
        "vision_wards": [vision_wards],
        "detector_wards": [detector_wards],
        "vision_score": [vision_score],
        "total_damagedealt": [total_damagedealt],
        "total_damagedealttochampions": [total_damagedealttochampions],
        "physical_damagedealt": [physical_damagedealt],
        "physical_damagedealttochampions": [physical_damagedealttochampions],
        "magic_damagedealt": [magic_damagedealt],
        "magic_damagedealttochampions": [magic_damagedealttochampions],
        "damagedealt_toobjectives": [damagedealt_toobjectives],
        "damagedealt_toturrets": [damagedealt_toturrets],
        "true_damagedealt": [true_damagedealt],
        "true_damagedealttochampions": [true_damagedealttochampions],
        "total_damagetaken": [total_damagetaken],
        "physical_damagetaken": [physical_damagetaken],
        "magic_damagetaken": [magic_damagetaken],
        "true_damagetaken": [true_damagetaken],
        "damage_selfmitigated": [damage_selfmitigated],
        "total_damagehieldedonteammates": [total_damagehieldedonteammates],
        "total_heal": [total_heal],
        "total_healsonteammates": [total_healsonteammates],
        "time_ccothers": [time_ccothers],
        "total_timespentdead": [total_timespentdead],
        "longest_timespentliving": [longest_timespentliving],
        "dragon_kills": [dragon_kills],
        "obj_stolen": [obj_stolen],
        "obj_stolenass": [obj_stolenass],
        "baron_kills": [baron_kills],
        "first_towerkill": [first_towerkill],
        "first_towerass": [first_towerass],
        "inhib_kills": [inhib_kills],
        "inhib_lost": [inhib_lost],
        "first_bloodkills": [first_bloodkills],
        "firstblood_assist": [firstblood_assist],
        "killing_sprees": [killing_sprees],
        "largest_killingsprees": [largest_killingsprees],
        "triple_kills": [triple_kills],
        "quadra_kills": [quadra_kills],
        "penta_kills": [penta_kills],
        "primary": [primary],
        "secondary": [secondary],
        "defense": [defense],
        "offense": [offense],
        "flex": [flex],
        "primary_style": [primary_style],
        "secondary_style": [secondary_style],
        "primary_keystone": [primary_keystone],
        "primary_perk_1": [primary_perk_1],
        "primary_perk_2": [primary_perk_2],
        "primary_perk_3": [primary_perk_3],
        "secondary_perk_1": [secondary_perk_1],
        "secondary_perk_2": [secondary_perk_2],
    })
    
    return match_df

In [48]:
# Processing my  Match History

match_ids = get_matchhistory(region="americas", puuid="lh_MFV9FRF3M2Apr3J6jnW80PiQKoJoc3SHJCJ_Jk_Y2WU-Iao60QBwtmeXXVTyFWDuHJ_YLizhQWQ")
df = pd.DataFrame()


for id in match_ids:
    game = get_matchdata(region="americas", match_id=id)
    match_df = process_match_json(game, puuid="lh_MFV9FRF3M2Apr3J6jnW80PiQKoJoc3SHJCJ_Jk_Y2WU-Iao60QBwtmeXXVTyFWDuHJ_YLizhQWQ")

    df = pd.concat([df, match_df])

In [49]:
# Runes Matching

perk = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json"
perk_styles = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json"

perk_json = requests.get(perk).json()
perkstyles_json = requests.get(perk_styles).json()

perk_ids = json_extract(perk_json,"id")
perk_names = json_extract(perk_json,"name")
perk_dict = dict(map(lambda i, j: (int(i), j), perk_ids, perk_names))

perkstyles_ids = json_extract(perkstyles_json,"id")
perkstyles_names = json_extract(perkstyles_json,"name")
perk_stylesdict = dict(map(lambda i, j: (int(i), j), perkstyles_ids, perkstyles_names))

# Item Matching

items ="https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json"
items_json = requests.get(items).json()
item_ids = json_extract(items_json,"id")
item_names = json_extract(items_json,"name")
items_dict = dict(map(lambda i, j: (int(i), j), item_ids, item_names))

# Summoner Spells 

sums = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/summoner-spells.json"
sums_json = requests.get(sums).json()
sum_ids = json_extract(sums_json,"id")
sum_names = json_extract(sums_json,"name")
sums_dict = dict(map(lambda i, j: (int(i), j), sum_ids, sum_names))


In [50]:
# Display Options

pd.options.display.max_columns = 100
pd.options.display.max_rows = 5

In [51]:
# Replace Dataframe Items
df.replace(sums_dict).replace(perk_dict).replace(perk_stylesdict).replace(items_dict)

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_time_stamp,patch,result,ifearly_surrender,if_surrender,assists,kills,deaths,minion_cs,jgl_cs,total_cs,gold_earned,gold_spent,items_purchased,item0,item1,item2,item3,item4,item5,item6,allin_ping,assistme_ping,enemymissing_ping,enemyvision_ping,needvision_ping,retreat_ping,defend_ping,coming_ping,push_ping,blue_ping,all_pings,spell1_casts,spell2_casts,spell3_casts,spell4_casts,summoner1_casts,summoner1_id,summoner2_casts,summoner2_id,wards_killed,wards_placed,sight_wards,vision_wards,detector_wards,vision_score,total_damagedealt,total_damagedealttochampions,physical_damagedealt,physical_damagedealttochampions,magic_damagedealt,magic_damagedealttochampions,damagedealt_toobjectives,damagedealt_toturrets,true_damagedealt,true_damagedealttochampions,total_damagetaken,physical_damagetaken,magic_damagetaken,true_damagetaken,damage_selfmitigated,total_damagehieldedonteammates,total_heal,total_healsonteammates,time_ccothers,total_timespentdead,longest_timespentliving,dragon_kills,obj_stolen,obj_stolenass,baron_kills,first_towerkill,first_towerass,inhib_kills,inhib_lost,first_bloodkills,firstblood_assist,killing_sprees,largest_killingsprees,triple_kills,quadra_kills,penta_kills,primary,secondary,defense,offense,flex,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
0,NA1_5248182232,[fYvK555yqRvFh2EMRFqEq9OxQ-ywHMo16kVX5yhz_zJ9O...,1742092659513,1924,1742092707688,1742094631713,15.5.662.6652,False,False,False,8,2,Exhaust,74,0,74,8419,7175,19,Zaz'Zak's Realmspike,Ionian Boots of Lucidity,Seraph's Embrace,Blackfire Torch,0,0,Oracle Lens,0,Cleanse,9,Ghost,0,8,0,Exhaust,0,Cleanse,22,73,To the King!,59,10,2,Flash,Ghost,Heal,2,Barrier,0,Cleanse,Cleanse,Placeholder,70536,18637,6456,1342,61517,16555,563,563,2561,739,15756,2330,11542,1883,7772,2012,3890,1188,46,104,Cloth Armor,0,0,0,0,False,False,0,Exhaust,False,False,Cleanse,2,0,0,0,"{'description': 'primaryStyle', 'selections': ...","{'description': 'subStyle', 'selections': [{'p...",Health Scaling,Attack Speed,Adaptive Force,Precision,Inspiration,Hail of Blades,Cheap Shot,Sixth Sense,Ultimate Hunter,Biscuit Delivery,Approach Velocity
0,NA1_5245819195,[0n4Djnoxe9Xw8NGzPrvb-FRir2SV3o5NqbNni0TN1_K00...,1741740524460,1990,1741740561282,1741742551339,15.5.662.6652,True,False,False,18,Ghost,,65,0,65,11994,11500,26,Bloodsong,Blade of The Ruined King,Berserker's Greaves,Mortal Reminder,Phantom Dancer,Dagger,Oracle Lens,0,0,Flash,,0,2,0,Flash,0,Cleanse,Smite,Ignite,86,15,Ignite,Exhaust,Flash,Flash,Heal,Smite,23,0,2,2,61,75412,19055,67757,15243,3464,2923,13466,2302,4190,888,21328,10290,10522,514,9233,0,4873,712,45,152,846,0,0,0,0,False,False,0,0,False,False,2,Exhaust,0,0,0,"{'description': 'primaryStyle', 'selections': ...","{'description': 'subStyle', 'selections': [{'p...",Health Scaling,Attack Speed,Adaptive Force,Precision,Inspiration,Hail of Blades,Cheap Shot,Sixth Sense,Ultimate Hunter,Biscuit Delivery,Approach Velocity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NA1_5234845486,[SsV8lYeXus-GGIgjkU0yhnbvmPUAjkhY5HKU7V1p7ikf4...,1740195633658,1851,1740195652057,1740197503137,15.4.657.5386,False,False,False,Heal,Exhaust,Heal,67,0,67,Steel Tempest,9225,Barrier,Zaz'Zak's Realmspike,Seraph's Embrace,Liandry's Torment,Ionian Boots of Lucidity,Amplifying Tome,Blasting Wand,Oracle Lens,Cleanse,0,0,2,0,0,0,2,0,Cleanse,Flash,91,42,78,8,,Flash,Heal,Heal,Cleanse,16,0,Cleanse,Cleanse,34,82489,16174,6486,1789,74935,14385,904,904,1067,0,25691,15421,9646,623,16165,1804,4616,1890,To the King!,206,626,0,0,0,